In [2]:
import random
import json
from datetime import datetime, timedelta

# Constants
COLLEGES = [
    "College of Engineering", "College of Science", "College of Arts",
    "College of Medicine", "College of Business", "College of Law"
]
DEPARTMENTS = [f"Department {i+1}" for i in range(28)]
VENUE_CAPACITIES = list(range(50, 321, 10))  # 50 to 320 in steps of 10
VENUES = [f"Venue {i+1}" for i in range(30)]
BLOCKS = [f"Block {chr(65 + i)}" for i in range(10)]  # Block A to Block J
DATES = [
    (datetime(2024, 7, 22) + timedelta(days=i)).strftime("%Y-%m-%d")
    for i in range(20) if (i % 7) not in [5, 6]  # Exclude Sundays
]
INVIGILATORS = [f"Invigilator {i+1}" for i in range(100)]

TIME_SLOTS = [
    {"time_key": "08:00", "time_label": "08:00 AM - 10:30 AM"},
    {"time_key": "11:00", "time_label": "11:00 AM - 01:00 PM"},
    {"time_key": "14:00", "time_label": "02:00 PM - 04:00 PM"},
]

LUNCH_BREAK = "13:00"

def generate_data():
    data = {
        "colleges": [],
        "departments": [],
        "courses": [],
        "students_summary": [],
        "venues": [],
        "invigilators": [],
        "time_slots": {slot["time_key"]: slot["time_label"] for slot in TIME_SLOTS},
        "lunch_break": LUNCH_BREAK,
        "dates": DATES,
        "metadata": {
            "version": "v1.0",
            "generated_on": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
    }

    # Colleges and Departments
    for college in COLLEGES:
        college_departments = random.sample(DEPARTMENTS, 4)
        data["colleges"].append({"name": college, "departments": college_departments})
        data["departments"].extend(college_departments)

    # Courses
    for department in data["departments"]:
        for level in range(100, 500, 100):  # Level 100 to 400
            num_courses = random.randint(10, 14)
            courses = [
                f"{department} Course {level}-{i+1}" for i in range(num_courses)
            ]
            data["courses"].extend(
                [{"name": course, "level": level, "department": department} for course in courses]
            )

    # Students Summary
    for department in data["departments"]:
        students_per_level = {
            "level_100": random.randint(50, 200),
            "level_200": random.randint(50, 150),
            "level_300": random.randint(30, 120),
            "level_400": random.randint(20, 100)
        }
        data["students_summary"].append({"department": department, "students": students_per_level})

    # Venues
    for venue, capacity, block in zip(VENUES, VENUE_CAPACITIES, random.choices(BLOCKS, k=len(VENUES))):
        data["venues"].append({
            "name": venue,
            "capacity": capacity,
            "block": block
        })

    # Invigilators
    data["invigilators"] = [{"name": inv} for inv in INVIGILATORS]

    return data

# Save JSON file
with open("data8.json", "w") as f:
    json.dump(generate_data(), f, indent=4)

print("Data generated and saved to data8.json")


Data generated and saved to data8.json


In [3]:
import json
import pandas as pd
from datetime import datetime
from collections import defaultdict
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill

# Constants
TIME_SLOTS = {
    "08:00": "08:00 AM - 10:30 AM",
    "11:00": "11:00 AM - 01:00 PM",
    "14:00": "02:00 PM - 04:00 PM"
}
DAY_ABBREVIATIONS = {
    "Monday": "MON", "Tuesday": "TUE", "Wednesday": "WED",
    "Thursday": "THU", "Friday": "FRI", "Saturday": "SAT", "Sunday": "SUN"
}

# Load generated data
with open("data8.json", "r") as f:
    data = json.load(f)

# Helper Functions
def assign_invigilators(invigilators, num_needed=2):
    """Round-robin assign invigilators."""
    invigilator_pool = iter(invigilators)
    assigned = []
    for _ in range(num_needed):
        try:
            assigned.append(next(invigilator_pool))
        except StopIteration:
            invigiligator_pool = iter(invigilators)
            assigned.append(next(invigilator_pool))
    return assigned

def generate_timetable(data):
    """
    Generate the exam timetable based on the provided format.
    """
    timetable = []
    unscheduled_courses = []
    venue_usage = defaultdict(lambda: defaultdict(list))  # Track venue usage per day and time slot

    max_exams_per_day = len(data["venues"]) * len(TIME_SLOTS)
    current_day_index = 0

    for course in data["courses"]:
        department = course["department"]
        level = course["level"]
        course_name = course["name"]

        # Get student count
        student_count = next(
            (summary["students"].get(f"level_{level}", 0) for summary in data["students_summary"] if summary["department"] == department),
            0
        )

        assigned_time_slot = None
        venue = None

        while current_day_index < len(data["dates"]):
            current_day = data["dates"][current_day_index]
            day_of_week = datetime.strptime(current_day, "%Y-%m-%d").strftime("%A")
            day_abbr = DAY_ABBREVIATIONS[day_of_week]

            for time_key, time_label in TIME_SLOTS.items():
                venue = next(
                    (v for v in data["venues"] if v["name"] not in venue_usage[current_day][time_key] and v["capacity"] >= student_count),
                    None
                )
                if venue:
                    assigned_time_slot = time_label
                    venue_usage[current_day][time_key].append(venue["name"])
                    break

            if assigned_time_slot:
                break
            else:
                current_day_index += 1

        if not assigned_time_slot or not venue:
            unscheduled_courses.append({
                "Course": course_name,
                "Reason": "No available slots or venues",
                "Students Population": student_count
            })
            continue

        invigilators = assign_invigilators(data["invigilators"], num_needed=2)

        timetable.append({
            "Date": current_day,
            "Day": day_abbr,
            "Time": assigned_time_slot,
            "Course": course_name,
            "Venue": f"{venue['name']} [Block {venue['block']}, {venue['capacity']}]",
            "Invigilators": ", ".join(inv["name"] for inv in invigilators),
            "Students Population": student_count
        })

    return timetable, unscheduled_courses

def save_timetable_to_excel(timetable, unscheduled_courses, file_name="timetable.xlsx"):
    """
    Save the timetable and unscheduled courses to an Excel file, formatted for readability.
    """
    with pd.ExcelWriter(file_name, engine="openpyxl") as writer:
        # Save Timetable
        df_timetable = pd.DataFrame(timetable)
        df_timetable.to_excel(writer, index=False, sheet_name="Exam Timetable")

        workbook = writer.book
        worksheet = writer.sheets["Exam Timetable"]
        yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

        # Adjust column widths and highlight header
        for col_idx, col_name in enumerate(df_timetable.columns, start=1):
            column_letter = get_column_letter(col_idx)
            worksheet[f"{column_letter}1"].fill = yellow_fill
            max_length = max(df_timetable[col_name].astype(str).map(len).max(), len(col_name))
            worksheet.column_dimensions[column_letter].width = max_length + 2

        # Save Unscheduled Courses
        if unscheduled_courses:
            df_unscheduled = pd.DataFrame(unscheduled_courses)
            df_unscheduled.to_excel(writer, index=False, sheet_name="Unscheduled Courses")

            worksheet_unscheduled = writer.sheets["Unscheduled Courses"]
            for col_idx, col_name in enumerate(df_unscheduled.columns, start=1):
                column_letter = get_column_letter(col_idx)
                worksheet_unscheduled[f"{column_letter}1"].fill = yellow_fill
                max_length = max(df_unscheduled[col_name].astype(str).map(len).max(), len(col_name))
                worksheet_unscheduled.column_dimensions[column_letter].width = max_length + 2

    print(f"Timetable saved successfully to {file_name}")

# Execution
try:
    timetable, unscheduled_courses = generate_timetable(data)
    save_timetable_to_excel(timetable, unscheduled_courses)
except Exception as e:
    print(f"Error: {e}")


Timetable saved successfully to timetable.xlsx
